# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f992c116bb0>
├── 'a' --> tensor([[-0.6105, -0.8414,  1.3082],
│                   [-0.6045, -1.2510, -0.7964]])
└── 'x' --> <FastTreeValue 0x7f992c1168b0>
    └── 'c' --> tensor([[-0.1577, -1.3917, -0.1238,  1.5335],
                        [ 0.8482, -1.0928,  1.2896, -0.4015],
                        [ 0.0902,  0.1388, -0.9605,  1.1112]])

In [4]:
t.a

tensor([[-0.6105, -0.8414,  1.3082],
        [-0.6045, -1.2510, -0.7964]])

In [5]:
%timeit t.a

62.6 ns ± 0.0806 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f992c116bb0>
├── 'a' --> tensor([[ 1.6538, -0.0967, -0.0763],
│                   [ 1.6051, -0.1677, -0.6941]])
└── 'x' --> <FastTreeValue 0x7f992c1168b0>
    └── 'c' --> tensor([[-0.1577, -1.3917, -0.1238,  1.5335],
                        [ 0.8482, -1.0928,  1.2896, -0.4015],
                        [ 0.0902,  0.1388, -0.9605,  1.1112]])

In [7]:
%timeit t.a = new_value

70.7 ns ± 0.0682 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6105, -0.8414,  1.3082],
               [-0.6045, -1.2510, -0.7964]]),
    x: Batch(
           c: tensor([[-0.1577, -1.3917, -0.1238,  1.5335],
                      [ 0.8482, -1.0928,  1.2896, -0.4015],
                      [ 0.0902,  0.1388, -0.9605,  1.1112]]),
       ),
)

In [10]:
b.a

tensor([[-0.6105, -0.8414,  1.3082],
        [-0.6045, -1.2510, -0.7964]])

In [11]:
%timeit b.a

58.4 ns ± 0.117 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3007,  2.8622,  1.5727],
               [ 2.1968,  1.3861,  1.4788]]),
    x: Batch(
           c: tensor([[-0.1577, -1.3917, -0.1238,  1.5335],
                      [ 0.8482, -1.0928,  1.2896, -0.4015],
                      [ 0.0902,  0.1388, -0.9605,  1.1112]]),
       ),
)

In [13]:
%timeit b.a = new_value

513 ns ± 0.158 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

865 ns ± 1.07 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 26 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 225 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 693 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f98889ffd90>
├── 'a' --> tensor([[[-0.6105, -0.8414,  1.3082],
│                    [-0.6045, -1.2510, -0.7964]],
│           
│                   [[-0.6105, -0.8414,  1.3082],
│                    [-0.6045, -1.2510, -0.7964]],
│           
│                   [[-0.6105, -0.8414,  1.3082],
│                    [-0.6045, -1.2510, -0.7964]],
│           
│                   [[-0.6105, -0.8414,  1.3082],
│                    [-0.6045, -1.2510, -0.7964]],
│           
│                   [[-0.6105, -0.8414,  1.3082],
│                    [-0.6045, -1.2510, -0.7964]],
│           
│                   [[-0.6105, -0.8414,  1.3082],
│                    [-0.6045, -1.2510, -0.7964]],
│           
│                   [[-0.6105, -0.8414,  1.3082],
│                    [-0.6045, -1.2510, -0.7964]],
│           
│                   [[-0.6105, -0.8414,  1.3082],
│                    [-0.6045, -1.2510, -0.7964]]])
└── 'x' --> <FastTreeValue 0x7f98889ff6a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 42.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9881e39cd0>
├── 'a' --> tensor([[-0.6105, -0.8414,  1.3082],
│                   [-0.6045, -1.2510, -0.7964],
│                   [-0.6105, -0.8414,  1.3082],
│                   [-0.6045, -1.2510, -0.7964],
│                   [-0.6105, -0.8414,  1.3082],
│                   [-0.6045, -1.2510, -0.7964],
│                   [-0.6105, -0.8414,  1.3082],
│                   [-0.6045, -1.2510, -0.7964],
│                   [-0.6105, -0.8414,  1.3082],
│                   [-0.6045, -1.2510, -0.7964],
│                   [-0.6105, -0.8414,  1.3082],
│                   [-0.6045, -1.2510, -0.7964],
│                   [-0.6105, -0.8414,  1.3082],
│                   [-0.6045, -1.2510, -0.7964],
│                   [-0.6105, -0.8414,  1.3082],
│                   [-0.6045, -1.2510, -0.7964]])
└── 'x' --> <FastTreeValue 0x7f9888a142e0>
    └── 'c' --> tensor([[-0.1577, -1.3917, -0.1238,  1.5335],
                        [ 0.8482, -1.0928,  1.2896, -0.4015],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.9 µs ± 93.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.6105, -0.8414,  1.3082],
                [-0.6045, -1.2510, -0.7964]],
       
               [[-0.6105, -0.8414,  1.3082],
                [-0.6045, -1.2510, -0.7964]],
       
               [[-0.6105, -0.8414,  1.3082],
                [-0.6045, -1.2510, -0.7964]],
       
               [[-0.6105, -0.8414,  1.3082],
                [-0.6045, -1.2510, -0.7964]],
       
               [[-0.6105, -0.8414,  1.3082],
                [-0.6045, -1.2510, -0.7964]],
       
               [[-0.6105, -0.8414,  1.3082],
                [-0.6045, -1.2510, -0.7964]],
       
               [[-0.6105, -0.8414,  1.3082],
                [-0.6045, -1.2510, -0.7964]],
       
               [[-0.6105, -0.8414,  1.3082],
                [-0.6045, -1.2510, -0.7964]]]),
    x: Batch(
           c: tensor([[[-0.1577, -1.3917, -0.1238,  1.5335],
                       [ 0.8482, -1.0928,  1.2896, -0.4015],
                       [ 0.0902,  0.1388, -0.9605,  1.1112]],
         

In [26]:
%timeit Batch.stack(batches)

77.3 µs ± 65.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.6105, -0.8414,  1.3082],
               [-0.6045, -1.2510, -0.7964],
               [-0.6105, -0.8414,  1.3082],
               [-0.6045, -1.2510, -0.7964],
               [-0.6105, -0.8414,  1.3082],
               [-0.6045, -1.2510, -0.7964],
               [-0.6105, -0.8414,  1.3082],
               [-0.6045, -1.2510, -0.7964],
               [-0.6105, -0.8414,  1.3082],
               [-0.6045, -1.2510, -0.7964],
               [-0.6105, -0.8414,  1.3082],
               [-0.6045, -1.2510, -0.7964],
               [-0.6105, -0.8414,  1.3082],
               [-0.6045, -1.2510, -0.7964],
               [-0.6105, -0.8414,  1.3082],
               [-0.6045, -1.2510, -0.7964]]),
    x: Batch(
           c: tensor([[-0.1577, -1.3917, -0.1238,  1.5335],
                      [ 0.8482, -1.0928,  1.2896, -0.4015],
                      [ 0.0902,  0.1388, -0.9605,  1.1112],
                      [-0.1577, -1.3917, -0.1238,  1.5335],
                      [ 0.8482, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

319 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
